## Importing Libraries

In [ ]:
import lightningchart as lc 
import pandas as pd
import numpy as np
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

lc.set_license("LICENSE-KEY")

## Accessing and Downloading Sensor Data

In [ ]:
# Initialize the client to fetch data from GFZ
client = Client("GFZ")

# Define the start and end time for the data retrieval
starttime = UTCDateTime("2016-07-12T10:00:00")
endtime = UTCDateTime("2016-07-12T11:00:00")

# Fetch waveforms for the specified station and time range
# Here we are fetching data for the vertical component (HHZ) only
st = client.get_waveforms(network="1C", station="WAR1", location="--", channel="HHZ", starttime=starttime, endtime=endtime)

# Select the first trace in the stream
tr = st[0]

# Print the metadata of the trace to inspect various attributes including the number of data points
print(tr.stats)

## Uptime of Each Sensor

In [ ]:
# Defining the uptime hours based on the abstract text provided in the source
uptime_hours = {
    'station': ['WAR1', 'WAR2', 'WAR3', 'WAR4', 'WAR5', 'WAR6', 'WAR7', 'WAR8', 'WAR9'],
    'uptime_hours': [41 * 24, 49 * 24, 49 * 24, 49 * 24, 49 * 24, 49 * 24, 47 * 24, 49 * 24, 49 * 24],
    'total_hours': 49 * 24
}

# Creating a DataFrame from the uptime_hours dictionary
uptime_df = pd.DataFrame(uptime_hours)

# Calculating the uptime percentage for each station
uptime_df['uptime_percentage'] = (uptime_df['uptime_hours'] / uptime_df['total_hours']) * 100

# Preparing the data for the LightningChart bar chart
# The data is structured as a list of dictionaries, each containing 'category' and 'value' keys
data = [{'category': station, 'value': uptime} for station, uptime in zip(uptime_df['station'], uptime_df['uptime_percentage'])]

# Creating a vertical bar chart with a dark theme
chart = lc.BarChart(
    vertical=True,
    theme=lc.Themes.Dark,
    title='Sensor Uptime Percentage by Station'
)
# Disabling sorting of the categories in the chart
chart.set_sorting('disabled')
# Setting the data for the bar chart
chart.set_data(data)

# Open the chart
chart.open()

## Visualization of a Single Sensor’s Raw Waveform Data

In [ ]:
# Get the time values in seconds from the start of the trace
x_values_seconds = st[0].times().tolist()

# Convert the start time to milliseconds
start_time = st[0].stats.starttime.timestamp * 1000

# If the time is in UTC+3, subtract 3 hours (10800 seconds) to adjust the time values
offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]

# Get the data values (raw counts) for the trace
y_values = st[0].data.tolist()

# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Seismic Data from WAR1"
)
# Remove the default x-axis
chart.get_default_x_axis().dispose()

x_axis = chart.add_x_axis(axis_type='linear-highPrecision') # Add a high-precision linear x-axis
x_axis.set_tick_strategy('DateTime')    # Set the x-axis to display date-time value
x_axis.set_scroll_strategy('progressive')   # Set the x-axis to use a progressive scroll strategy
x_axis.set_title("Time")    # Set the title for the x-axis

# Set the title for the y-axis
chart.get_default_y_axis().set_title("Raw count")

# Add a line series to the chart and append the x and y values
series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

# Set the line thickness for better visibility
series.set_line_thickness(2)

# Open the chart
chart.open()

### Waveform Data for all Components (HHZ, HHE, HHN)

In [ ]:
# Initialize the client to fetch data from GFZ
client = Client("GFZ")

# Define the start and end time for the data retrieval
# We are using the previously used start and end times
starttime = starttime
endtime = endtime
station = "WAR1"

# Fetch waveforms for all three components (Z, E, N) from the specified station and time range
st = client.get_waveforms(network="1C", station=station, location="--", channel="HH?", starttime=starttime, endtime=endtime)
st.merge(method=1)

# Separate the components into individual traces
tr_z = st.select(channel="HHZ")[0]
tr_e = st.select(channel="HHE")[0]
tr_n = st.select(channel="HHN")[0]

# Calculate time values for the traces and adjust for UTC+3
times_seconds = tr_z.times().tolist()  # Get the times in seconds
start_time_ms = tr_z.stats.starttime.timestamp * 1000  # Convert start time to milliseconds
offset_seconds = 3 * 3600  # Offset for UTC+3 in seconds
times = [start_time_ms + (sec - offset_seconds) * 1000 for sec in times_seconds]  # Adjust times

# Get data values for each component
data_z = tr_z.data.tolist()
data_e = tr_e.data.tolist()
data_n = tr_n.data.tolist()

# Create a chart with dark theme
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Seismic Waveform for All WAR1 Components'
)

# Remove the default x-axis and set up a custom one
chart.get_default_x_axis().dispose()
chart.get_default_y_axis().set_title("Raw count")

x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')  # Set the x-axis to display date-time values
x_axis.set_scroll_strategy('progressive')
x_axis.set_title("Time")

# Add waveform series for each component to the chart
waveform_series_z = chart.add_line_series().append_samples(x_values=times, y_values=data_z).set_name('HHZ').set_line_thickness(2)
waveform_series_e = chart.add_line_series().append_samples(x_values=times, y_values=data_e).set_name('HHE').set_line_thickness(2)
waveform_series_n = chart.add_line_series().append_samples(x_values=times, y_values=data_n).set_name('HHN').set_line_thickness(2)

# Open the chart
chart.open()

## Multiple Sensors’ Data Visualization (Stacked Chart)

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

# List of stations
stations = [f"WAR{i}" for i in range(1, 10)]

# Dictionary to hold data
data = {}

# Fetch waveforms for each station and store in the data dictionary
for station in stations:
        st = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime, endtime=endtime)
        data[station] = st[0]

# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Seismic Data Comparison (WAR1 to WAR9)"
)

# Remove the default x-axis and y-axis
chart.get_default_x_axis().dispose()
chart.get_default_y_axis().dispose()

# Add a high-precision linear x-axis
x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')    # Set the x-axis to display date-time values
x_axis.set_scroll_strategy('progressive')   # Set the x-axis to use a progressive scroll strategy
x_axis.set_title("Time")    # Set the title for the x-axis

# Create and add stacked y-axes and series for each station
for i, station in enumerate(stations):
    axis_y = chart.add_y_axis(stack_index=i)    # Add a new y-axis for each station
    axis_y.set_margins(15 if i > 0 else 0, 15 if i < len(stations) - 1 else 0)  # Set margins for y-axes
    axis_y.set_title(title=station) # Set the title for the y-axis

    # Get the trace data for the current station
    trace = data[station]
    x_values_seconds = trace.times().tolist()   # Get time values in seconds from the start of the trace
    start_time = trace.stats.starttime.timestamp * 1000  # Convert the start time to milliseconds
    offset_seconds = 3 * 3600  # Offset for UTC+3
    x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds] # Adjust the time values
    y_values = trace.data.tolist()  # Get the data values (raw counts) for the trace

    # Add a line series for each station
    series = chart.add_line_series(y_axis=axis_y, data_pattern='ProgressiveX')
    series.append_samples(x_values, y_values)   # Append the x and y values to the series
    
    series.set_line_thickness(2)    # Set the line thickness for better visibility
    
# Open the chart
chart.open()

## Velocity Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

# Fetch waveforms for the specified station and time range
stVel = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information for the specified station and channel
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

# Remove the instrument response to obtain velocity data
stVel.remove_response(inventory=inv, output="VEL")

# Get the time values in seconds from the start of the trace
x_values_seconds = stVel[0].times().tolist()
start_time = stVel[0].stats.starttime.timestamp * 1000  # Convert the start time to milliseconds

 # If the time is in UTC+3, subtract 3 hours (10800 seconds)
offset_seconds = 3 * 3600 
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]

# Get the data values (velocity) for the trace
y_values = stVel[0].data.tolist()

# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Velocity Data from WAR1"
)

# Remove the default x-axis
chart.get_default_x_axis().dispose()

# Set the title for the y-axis
chart.get_default_y_axis().set_title("Velocity (m/s)")

# Add a high-precision linear x-axis
x_axis = chart.add_x_axis(axis_type='linear-highPrecision')
x_axis.set_tick_strategy('DateTime')    # Set the x-axis to display date-time values
x_axis.set_scroll_strategy('progressive')   # Set the x-axis to use a progressive scroll strategy
x_axis.set_title("Time")    # Set the title for the x-axis

# Add a line series to the chart and append the x and y values
series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

# Set the line thickness for better visibility
series.set_line_thickness(2)

# Open the chart
chart.open()

## Displacement Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

# Fetch waveforms for the specified station and time range
stDis = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information for the specified station and channel
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

# Remove the instrument response to obtain displacement data
stDis.remove_response(inventory=inv, output="DISP")

x_values_seconds = stDis[0].times().tolist()
start_time = stDis[0].stats.starttime.timestamp * 1000

offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
y_values = stDis[0].data.tolist()   # Get the data values (displacement) for the tra

# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Displacement Data from WAR1"
)

# Remove the default x-axis
chart.get_default_x_axis().dispose()

# Set the title for the y-axis
chart.get_default_y_axis().set_title("Displacement (m)")

# Add a high-precision linear x-axis
x_axis = chart.add_x_axis(axis_type='linear-highPrecision')

# Set the x-axis to display date-time values
x_axis.set_tick_strategy('DateTime')

# Set the x-axis to use a progressive scroll strategy
x_axis.set_scroll_strategy('progressive')

# Set the title for the x-axis
x_axis.set_title("Time")

# Add a line series to the chart and append the x and y values
series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

# Set the line thickness for better visibility
series.set_line_thickness(2)

# Open the chart
chart.open()

## Acceleration Visualization

In [ ]:
client = Client("GFZ")
starttime = starttime
endtime = endtime

# Fetch waveforms for the specified station and time range
stAcc = client.get_waveforms(network="1C", station="WAR1", location="", channel="HHZ", starttime=starttime, endtime=endtime)

# Fetch response information for the specified station and channel
inv = client.get_stations(network="1C", station="WAR1", location="", channel="HHZ", level="response")

# Remove the instrument response to obtain acceleration data
stAcc.remove_response(inventory=inv, output="ACC")

# Get the time values in seconds from the start of the trace
x_values_seconds = stAcc[0].times().tolist()

# Convert the start time to milliseconds
start_time = stAcc[0].stats.starttime.timestamp * 1000

# If the time is in UTC+3, subtract 3 hours (10800 seconds) to adjust the time values
offset_seconds = 3 * 3600
x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]

# Get the data values (acceleration) for the trace
y_values = stAcc[0].data.tolist()

# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title="Acceleration Data from WAR1"
)

# Remove the default x-axis
chart.get_default_x_axis().dispose()

# Set the title for the y-axis
chart.get_default_y_axis().set_title("Acceleration (m/s²)")

# Add a high-precision linear x-axis
x_axis = chart.add_x_axis(axis_type='linear-highPrecision')

# Set the x-axis to display date-time values
x_axis.set_tick_strategy('DateTime')

# Set the x-axis to use a progressive scroll strategy
x_axis.set_scroll_strategy('progressive')

# Set the title for the x-axis
x_axis.set_title("Time")

# Add a line series to the chart and append the x and y values
series = chart.add_line_series().append_samples(
    x_values=x_values,
    y_values=y_values
)

# Set the line thickness for better visibility
series.set_line_thickness(2)

# Open the chart
chart.open()

## Waveform Data + Velocity, Displacement and Acceleration Visualization (Dashboard)

In [ ]:
# Create a dashboard with 2 columns and 2 rows, using a dark theme
dashboard = lc.Dashboard(columns=2, rows=2, theme=lc.Themes.Dark)

# Common time offset for UTC+3
offset_seconds = 3 * 3600

# Helper function to calculate x_values adjusted for the offset
def calculate_x_values(stream, offset_seconds):
    x_values_seconds = stream[0].times().tolist()
    start_time = stream[0].stats.starttime.timestamp * 1000
    x_values = [start_time + (sec - offset_seconds) * 1000 for sec in x_values_seconds]
    return x_values

# Calculate x_values for all data series
x_values0 = calculate_x_values(st, offset_seconds)     # Seismic waveform data
x_values1 = calculate_x_values(stVel, offset_seconds)  # Velocity data
x_values2 = calculate_x_values(stDis, offset_seconds)  # Displacement data
x_values3 = calculate_x_values(stAcc, offset_seconds)  # Acceleration data

# Prepare the y_values for all data series
y_values0 = st[0].data.tolist()      # Seismic waveform data
y_values1 = stVel[0].data.tolist()   # Velocity data
y_values2 = stDis[0].data.tolist()   # Displacement data
y_values3 = stAcc[0].data.tolist()   # Acceleration data

# Create a chart for seismic waveform data and add it to the dashboard
chart0 = dashboard.ChartXY(column_index=0, row_index=0, column_span=1, row_span=1, title="Seismic Waveform Data from WAR1")
chart0.get_default_x_axis().dispose()
chart0.get_default_y_axis().set_title("Raw count")

x_axis0 = chart0.add_x_axis(axis_type='linear-highPrecision')
x_axis0.set_tick_strategy('DateTime')
x_axis0.set_scroll_strategy('progressive')
x_axis0.set_title("Time")

series0 = chart0.add_line_series().append_samples(
    x_values=x_values0,
    y_values=y_values0
)
series0.set_line_thickness(2)

# Create a chart for velocity data and add it to the dashboard
chart1 = dashboard.ChartXY(column_index=1, row_index=0, column_span=1, row_span=1, title="Velocity Data from WAR1")
chart1.get_default_x_axis().dispose()
chart1.get_default_y_axis().set_title("Velocity (m/s)")

x_axis1 = chart1.add_x_axis(axis_type='linear-highPrecision')
x_axis1.set_tick_strategy('DateTime')
x_axis1.set_scroll_strategy('progressive')
x_axis1.set_title("Time")

series1 = chart1.add_line_series().append_samples(
    x_values=x_values1,
    y_values=y_values1
)
series1.set_line_thickness(2)

# Create a chart for displacement data and add it to the dashboard
chart2 = dashboard.ChartXY(column_index=0, row_index=1, column_span=1, row_span=1, title="Displacement Data from WAR1")
chart2.get_default_x_axis().dispose()
chart2.get_default_y_axis().set_title("Displacement (m)")

x_axis2 = chart2.add_x_axis(axis_type='linear-highPrecision')
x_axis2.set_tick_strategy('DateTime')
x_axis2.set_scroll_strategy('progressive')
x_axis2.set_title("Time")

series2 = chart2.add_line_series().append_samples(
    x_values=x_values2,
    y_values=y_values2
)
series2.set_line_thickness(2)

# Create a chart for acceleration data and add it to the dashboard
chart3 = dashboard.ChartXY(column_index=1, row_index=1, column_span=1, row_span=1, title="Acceleration Data from WAR1")
chart3.get_default_x_axis().dispose()
chart3.get_default_y_axis().set_title("Acceleration (m/s²)")

x_axis3 = chart3.add_x_axis(axis_type='linear-highPrecision')
x_axis3.set_tick_strategy('DateTime')
x_axis3.set_scroll_strategy('progressive')
x_axis3.set_title("Time")

series3 = chart3.add_line_series().append_samples(
    x_values=x_values3,
    y_values=y_values3
)
series3.set_line_thickness(2)

# Open the dashboard
dashboard.open()

## Additional Visualizations

### Area Chart

In [ ]:
# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Seismic Activity Over Time for Three Sensors'
)

# Initialize the client to fetch data from GFZ
client = Client("GFZ")
starttime = starttime
endtime = endtime

# List of stations and corresponding colors for plotting
stations = ["WAR1", "WAR2", "WAR4"]
colors = [(255, 215, 0), (0, 255, 255), (255, 0, 255)]  # Yellow for WAR1, cyan for WAR2, magenta for WAR4

# Loop through each station to fetch data and add it to the chart
for i, station in enumerate(stations):
    # Fetch waveforms for the specified station and time range
    st = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime, endtime=endtime)
    tr = st[0]  # Select the first trace in the stream

    # Get time and data values from the trace
    times = tr.times()
    data = tr.data

    # Add an area series to the chart for the current station
    series = chart.add_area_series()
    series.add(times, data)
    series.set_name(f'{station} Activity')  # Set the name for the series
    series.set_line_color(lc.Color(*colors[i]))  # Set the line color for the series

# Open the chart
chart.open()

### Bipolar Chart

In [ ]:
# Create a chart with a dark theme and title
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Comparison of Seismic Activity'
)

# Initialize the client to fetch data from GFZ
client = Client("GFZ")

# Define the start and end times for two different time ranges
starttime1 = UTCDateTime("2016-07-12T09:30:00")
endtime1 = UTCDateTime("2016-07-12T10:30:00")
starttime2 = UTCDateTime("2016-07-12T10:30:00")
endtime2 = UTCDateTime("2016-07-12T11:30:00")
station = "WAR1"

# Fetch waveforms for the specified station and two different time ranges
st1 = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime1, endtime=endtime1)
st2 = client.get_waveforms(network="1C", station=station, location="--", channel="HHZ", starttime=starttime2, endtime=endtime2)

# Select the first trace in each stream
tr1 = st1[0]
tr2 = st2[0]

# Get time and data values from both traces
times1 = tr1.times()
data1 = tr1.data

times2 = tr2.times()
data2 = tr2.data

# Ensure both datasets have the same length by trimming the longer one
min_length = min(len(data1), len(data2))
data1 = data1[:min_length]
data2 = data2[:min_length]
times = times1[:min_length]

# Add a bipolar area series to the chart to show the difference between the two datasets
series = chart.add_bipolar_area_series().add(times, data1 - data2)
series.set_name('Seismic Activity Difference')  # Set the name for the series

# Open the chart
chart.open()